#### 【NGS解析】Arabidopsis RNA-Seq データの DGE解析に向けた前処理   <br>&nbsp; RNAseq_01  : Stepwise Pipeline

--------------------------------------------------------------------------</br>

  このノートブックはGoogle Colab上で動かすことを想定しており、  <br>
また、１セルずつ実行して１処理ずつ確認しながら進めていく形になっています。  

✅Arabidopsis thaliana の<strong>single-end</strong> RNA-Seq データを使用した、  <br>&nbsp;&nbsp; &nbsp;&nbsp;DGE解析に向けたデータの前処理です。  
✅Google Drive フォルダに次のファイルが入れてある前提です。
* drive_path :   accession_list.txt, param1~4.txt
* common_ref :   araport11.gtf, TAIR10_chr_all.fas, smr_v4.3_default_db.fasta  
<small>💡バックスラッシュ＼が円マーク￥で表示される場合はブラウザのフォントを英字フォントに変更する</small>  

※詳細はReadmeに記載しています。  
--------------------------------------------------------------------------</br>


🛠️ 初期設定：Google Drive をマウント（セッション再接続ごとに実行）

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
drive_path = "/content/drive/MyDrive/ngs_analysis/rna-seq_01"  # データを退避する先のパスをpython変数で定義
common_ref = "/content/drive/MyDrive/ngs_analysis/common_ref"    #  参照ファイルが保存してあるパスをpython変数で定義

 🛠️ 初期設定：Micromambaをインストール（セッション再接続ごとに実行）

In [ ]:
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj -C /root/ --strip-components=1

In [ ]:
import os

In [ ]:
os.environ["PATH"] = os.path.abspath("/root/") + os.pathsep  + os.environ["PATH"] # 環境変数に/root/パスを追加
print(os.environ["PATH"])

In [ ]:
!micromamba create -y -n ngs  # NGS用の仮想環境を作成

In [ ]:
os.environ["PATH"] = os.path.abspath("/root/.local/share/mamba/envs/ngs/bin") + os.pathsep  + os.environ["PATH"]  # 環境変数にパスを追加
print(os.environ["PATH"])

---
⬇️ 処理ここから ⬇️

■ 解析対象のリードデータを SRA 互換リポジトリから取得してFASTQ 形式に変換

In [ ]:
!mkdir sra_data

In [ ]:
!micromamba install -y -q -n ngs -c conda-forge -c bioconda  sra-tools

In [ ]:
!prefetch --option-file {drive_path}/accession_list.txt  -O /content/sra_data/

In [ ]:
!fasterq-dump  /content/sra_data/*  -O /content/

In [ ]:
!cp /content/*fastq  {drive_path}  # fastqファイルをGoogle Driveに退避

■ fastqデータの品質をチェック 【FastQC】

In [ ]:
!micromamba install -y -q -n ngs -c conda-forge -c bioconda fastqc

In [ ]:
!fastqc /content/*.fastq

In [ ]:
!cp /content/*.html  {drive_path}  # FastQC ReportをGoogle Driveに退避

---
⏸️ セッションの一時終了  <br>
<small>&nbsp;&nbsp; &nbsp;&nbsp;※接続オプションから「接続を解除して削除」を選択して現在のセッションを終了  

  ・Google Drive に保存された.htmlファイル（FastQC Report）をダウンロードし、ブラウザで開いて確認します。  <br>
  ・FastQC の結果を参考にしてトリミング方法を検討します。  <br></small>



&nbsp;&nbsp; &nbsp;&nbsp;<small>💡 セッション終了後すぐに再接続すると、リソースの解放処理が完了しておらず、環境に不整合が生じることがあるため、数分時間をあける。</small>

▶️ セッションの再開  <br>
<small> &nbsp;&nbsp; &nbsp;&nbsp;※接続オプションから「ホストされているランタイムに接続」を選択してセッションを再接続  </small><br>

<small>&nbsp;&nbsp;🎯以下のステップを再実行：  <br>
&nbsp;&nbsp; &nbsp;&nbsp;🛠️初期設定：Google Drive をマウント  <br>
&nbsp;&nbsp; &nbsp;&nbsp;🛠️初期設定：Micromambaをインストール</small>

■ リード配列をカスタムトリミング【cutadapt】&【sortMeRNA】

In [ ]:
!cp {drive_path}/*.fastq  /content/  #退避していたfastqファイルをセッションにコピー

ー cutadaptを用いた人工配列と低品質部位のトリミング

In [ ]:
!micromamba install -y -q -n ngs -c conda-forge -c bioconda cutadapt

In [ ]:
!cp {drive_path}/params*.txt /content/  # cutadapt用のオプションと引数を指定したファイルをセッションにコピー

In [ ]:
%%bash
PARAMS1=$(<params1.txt)  # for removing structurally invalid reads
PARAMS2=$(<params2.txt)  # for trimming Template Switching Oligo sequences
PARAMS3=$(<params3.txt)  # for trimming Nextera adapters and primer sequences
PARAMS4=$(<params4.txt)  # for quality filtering and tail trimming
                                              # used for fastq data from SmartSeq2+Nextera chemistry, NextSeq
for file in /content/*.fastq; do
     cutadapt   $PARAMS1  -o temp1  "$file"   > "${file%.fastq}_report1.txt"
     cutadapt   $PARAMS2  -o temp2   temp1 > "${file%.fastq}_report2.txt"
     cutadapt   $PARAMS3  -o temp3   temp2 > "${file%.fastq}_report3.txt"
     cutadapt   $PARAMS4  -o "${file%.fastq}_cut"  temp3 > "${file%.fastq}_report4.txt"
     rm temp1 temp2 temp3;
done
 # 💡 左のファイル一覧で出力ファイルが増えていくことで進捗を確認

ー sortMeRNAを用いたrRNAの除去処理

In [ ]:
!micromamba install -y -q -n ngs -c conda-forge -c bioconda sortmerna=4.3.7

In [ ]:
!cp {common_ref}/smr_v4.3_default_db.fasta  /content/

In [ ]:
!for file in /content/*_cut; do sortmerna -ref smr_v4.3_default_db.fasta -reads "$file" -workdir /content/smr_work/ -no-best -a 4 -fastx -other "${file%_cut}_cleaned" &&  rm /content/smr_work/kvdb/*; done

■ トリミング後のfastq データの品質をチェック 【FastQC】

In [ ]:
!micromamba install -y -q -n ngs -c conda-forge -c bioconda fastqc

In [ ]:
!fastqc /content/*_cleaned.fq

In [ ]:
!cp /content/*_cleaned.fq {drive_path}  # トリミング後のfastqファイル(拡張子無し)をGoogle Driveに退避
!cp /content/*.html {drive_path}         # FstQC ReportをGoogle Drive に退避

⏸️ セッションの一時終了  <br>
       &nbsp;&nbsp;  &nbsp;&nbsp; 結果の確認<br>
▶️ セッションの再開

■ リード配列を参照ゲノムにマッピング 【HISAT2】

In [ ]:
!cp  {drive_path}/*_cleaned.fq  /content/  #保存していたトリミング後fastqファイルをセッションにコピー

In [ ]:
!micromamba install -y -q -n ngs -c conda-forge -c bioconda hisat2

In [ ]:
!hisat2_extract_splice_sites.py {common_ref}/araport11.gtf > splice_sites.txt  #	スプライスサイト情報を抽出

In [ ]:
!hisat2_extract_exons.py {common_ref}/araport11.gtf > exons.txt  # エクソン情報を抽出

In [ ]:
!hisat2-build --ss splice_sites.txt --exon exons.txt   {common_ref}/TAIR10_chr_all.fas  at_index #参照ゲノムをもとにhisat2のインデックスファイルを作成

In [ ]:
!for file in *_cleaned.fq ; do hisat2 -x at_index -p 2  --phred33  -q -U "$file" -S "${file%_cleaned}_aligned.sam";done

In [ ]:
!cp /content/*.ht2 {common_ref}  #  HISAT2インデックス（splice site情報・exon情報含む）をGoogle Driveに保存

■ SAM→BAM変換 (テキストからバイナリへ) 【Samtools】
  

In [ ]:
!micromamba install -y -q -n ngs -c conda-forge -c bioconda samtools

In [ ]:
!for file in /content/*_aligned.sam; do samtools view -b "$file" -o "${file%.sam}.bam"; done  # SAM→BAM変換

In [ ]:
!for file in /content/*.bam; do samtools sort "$file" -o "${file%_aligned.bam}_sorted.bam"; done # ソート

In [ ]:
!for file in /content/*_sorted.bam; do samtools index "$file"; done  # BAMインデックスファイルを作成（Genome Viewerで使用）

In [ ]:
!cp /content/*sorted.bam  {drive_path}  # ソート済みBAMファイルをGoogle Driveに保存

In [ ]:
!cp /content/*sorted.bam.bai  {drive_path}  # BAMインデックスファイルをGoogle Driveに保存

■ マッピング結果に基づきリード数を定量化 【subread】

In [ ]:
!micromamba install -y -q -n ngs -c conda-forge -c bioconda subread

In [ ]:
!featureCounts -T 2  -t exon -g gene_id  -a {common_ref}/araport11.gtf -o counts.txt  /content/*_sorted.bam

In [ ]:
!cp /content/counts.txt  {drive_path}  # リードカウントファイルをGoogle Driveに保存

🔚このNotebookはここまで  <br>
&nbsp;&nbsp; &nbsp;&nbsp;作成したカウントマトリックス（counts.txt）を用いてDGE解析に進みます。  

※ rna-seq_02 および rna-seq_03 では、ファイルを一時保存せず、パイプ入力によるストリーミングで逐次処理を行っています。



---

